In [7]:
import pandas as pd
from pandas import json_normalize

In [8]:
df = pd.read_json("../data/LRE_corpus/LRE_corpus-0.1/corpora.json", lines=True)

In [9]:
df

,id_str,lre_annotation
0,527760967652749312,[]
1,527007061029056512,[]
2,528154012315426816,[]
3,528220003691024384,[]
4,526282301366226944,[]
...,...,...
9995,507167181210017792,[]
9996,529628482158878720,[]
9997,528209131945984000,[]
9998,521454722234851328,[]


In [10]:
# Proportion of datapoints where lre_annotation column is an empty list
df.lre_annotation.apply(lambda x: len(x) != 0).mean() * len(df)

793.0

In [11]:
def custom_function(value):
    if not value:
        return None
    # print(value)
    a = json_normalize(value)
    # print(a)
    a["entities"] = a["entities"].apply(lambda x: x[0]["geometry"]["coordinates"] if x else None)
    a["entities"] = a["entities"].apply(lambda x: tuple(x) if x and x[0] else None)
    a = a.rename(columns={"entities": "coordinates"})
    return a


result = df['lre_annotation'].apply(custom_function)


In [12]:
# Concatenate the resulting DataFrames
result = pd.concat([result[i] for i in range(len(result))], axis=0).reset_index(drop=True)
ids = [[id_s]*count for id_s, count in zip(df["id_str"], df["lre_annotation"].apply(len))]
result.insert(0, "id_str", sum(ids, []))
result

,id_str,start,end,surface,note,entityClass,coordinates
0,525988539553439744,19,24,ファミレス,,generic,None
1,527412572522160128,6,11,MOVIX,,facility,None
2,527412572522160128,24,26,利府,,location,"(140.97684, 38.33084)"
3,527412572522160128,34,39,MOVIX,,facility,None
4,527992658216488960,19,21,お店,複数の特定不能店舗,facility-UNSP,None
...,...,...,...,...,...,...,...
1039,521152818023907328,22,23,泉,,location-UNSP,None
1040,522292131923120128,0,3,アキバ,,facility,None
1041,529470205865373696,7,12,ラーメン屋,,facility-UNSP,None
1042,515384767269711872,33,36,森ビル,,facility,None


In [11]:
# proportion of results with non-null coordinates
result.coordinates.apply(lambda x: x is not None).mean() * len(result)

299.0

In [12]:
result["entityClass"].value_counts()

entityClass
location         298
facility-UNSP    273
facility         203
location-OOG      94
generic           65
facility-OOG      41
rail              25
fiction           24
location-UNSP     14
other              4
road               3
Name: count, dtype: int64

In [14]:
locationEntities = result[result["entityClass"] == "location"]

In [15]:
locationEntities.coordinates.apply(lambda x: x is not None).mean() * len(locationEntities)

298.0

# Retrieve tweets

In [1]:
import requests
import os
import json

bearer_token = "AAAAAAAAAAAAAAAAAAAAAM6%2BsgEAAAAAPRQOwv6bh4sIQSA1yFrSAHUTPwQ%3DIM4EBDYcompU0clp6dSfMRXQyLbdy4NC2NFeldkEpKxHEDRrzq"

def create_url():
    tweet_fields = "tweet.fields=lang,author_id"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    ids = "527412572522160128"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets?{}&{}".format(ids, tweet_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r


def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


url = create_url()
json_response = connect_to_endpoint(url)
print(json.dumps(json_response, indent=4, sort_keys=True))


403


Exception: Request returned an error: 403 {"client_id":"28491470","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Appropriate Level of API Access","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}